In [37]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from collections import defaultdict, Counter
from transformers import AutoTokenizer, BasicTokenizer, AutoModelForMaskedLM
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
import pytrec_eval

sys.path.append(os.pardir)
from splade_vocab.models import Splade, BEIRSpladeModel, BEIRSpladeModelIDF
from tqdm import tqdm_notebook

In [49]:
!export CUDA_VISIBLE_DEVICES=1
data_path = "/home/gaia_data/iida.h/BEIR/datasets/scidocs"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/25657 [00:00<?, ?it/s]

In [50]:
def search_result(model_path):
    model = Splade(model_path)
    model.eval()
    tokenizer = model.tokenizer
    beir_splade = BEIRSpladeModel(model, tokenizer)
    dres = DRES(beir_splade, batch_size=64)
    retriever = EvaluateRetrieval(dres, score_function="dot", )
    results = retriever.retrieve(corpus, queries)
    del retriever
    del dres
    del model
    return results

In [51]:
model_pathes = {"mlm-splade-62783": "/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/",
               "mlm-splade-30522": "/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/",
                "splade-62783": "/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model_init/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-init_model-raw-remove--62783-batch_size_40-2022-04-17_09-55-08/",
               "splade": "/home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/"}

In [52]:
tks = {"splade-62783": AutoTokenizer.from_pretrained(os.path.join(model_pathes["mlm-splade-62783"], "0_MLMTransformer")),
       "splade": AutoTokenizer.from_pretrained(os.path.join(model_pathes["splade"], "0_MLMTransformer")),
      "space": BasicTokenizer()}

# Subword Ratio

In [109]:
def word_subword_alignment(t_subs, t_words):
    word_subword = defaultdict(list)
    t_sub = t_subs.pop(0)
    while len(t_subs) and len(t_words):
        t_word = t_words.pop(0)
        while t_sub.strip("#") in t_word and len(t_words) and len(t_subs):
            if t_sub != t_word:
                word_subword[t_word].append(t_sub)
            t_sub = t_subs.pop(0)
            
    del_key = []
    for word, subs in word_subword.items():
        if subs[0][0] != word[0]:
            del_key.append(word)
            
    for word in del_key:
        del word_subword[word]
    return word_subword

In [105]:
corpus['4983']

{'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 versus 1.1 microm2/ms). Relative anisotropy was higher the closer birth was to term with 

In [113]:
def word_subword_ratio(corpus, tks):
    nume = 0
    denom = 0
    for cid, d_text in tqdm(corpus.items()):
        text = d_text["title"] + " " + d_text["text"]
        t_words = tks["space"].tokenize(text)
        t_subs_org = tks["splade"].tokenize(text)
        word_subword_splade = word_subword_alignment(t_subs_org, t_words)
        t_words = tks["space"].tokenize(text)
        t_subs_62783 = tks["splade-62783"].tokenize(text)
        word_subword_splade_62783 = word_subword_alignment(t_subs_62783, t_words)
        nume += len(word_subword_splade_62783)
        denom += len(word_subword_splade)
    print(nume, denom, (denom - nume) / denom)
        
        


In [114]:
word_subword_ratio(corpus, tks)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:29<00:00, 175.12it/s]

22911 105263 0.7823451735177602


# Embedding

In [305]:
def embed_move_w_splade(ini_model, splade_model):
    mlm_head_vec = ini_model.cls.predictions.decoder.weight
    mlm_sphead_vec = splade_model.transformer.cls.predictions.decoder.weight
    mlm_head_vec_norm = mlm_head_vec / torch.unsqueeze(torch.norm(mlm_head_vec, dim=1), 1)
    mlm_sphead_vec_norm = mlm_sphead_vec / torch.unsqueeze(torch.norm(mlm_sphead_vec, dim=1), 1)
    l2 = torch.norm(mlm_head_vec - mlm_sphead_vec, dim=1)
    l2_org = torch.mean(l2[:30522])
    l2_add = torch.mean(l2[30522:])
    print(f"l2: {torch.mean(l2)}, l2_org: {l2_org}, l2_add: {l2_add}")
    cos_sim = torch.sum(mlm_head_vec_norm * mlm_sphead_vec_norm, dim=1)
    cos_sim_org = torch.mean(cos_sim[:30522])
    cos_sim_add = torch.mean(cos_sim[30522:])
    print(f"cos_sim: {torch.mean(cos_sim)}, cos_sim_org: {cos_sim_org}, cos_sim_add: {cos_sim_add}")
    topk_l2, topk_ids = torch.topk(l2, 50)
    print(topk_l2)
    print(topk_ids)
    print(splade_model.tokenizer.decode(topk_ids))

In [301]:
def embed_move_w_bert(ini_model, mlm_model, tokenizer):
    ini_head_vec = ini_model.cls.predictions.decoder.weight
    mlm_head_vec = mlm_model.cls.predictions.decoder.weight
    ini_head_vec_norm = ini_head_vec / torch.unsqueeze(torch.norm(ini_head_vec, dim=1), 1)
    mlm_head_vec_norm = mlm_head_vec / torch.unsqueeze(torch.norm(mlm_head_vec, dim=1), 1)
    l2 = torch.norm(ini_head_vec - mlm_head_vec, dim=1)
    l2_org = torch.mean(l2[:30522])
    l2_add = torch.mean(l2[30522:])
    print(f"l2_org: {l2_org}, l2_add: {l2_add}")
    cos_sim = torch.sum(ini_head_vec_norm * mlm_head_vec_norm, dim=1)
    cos_sim_org = torch.mean(cos_sim[:30522])
    cos_sim_add = torch.mean(cos_sim[30522:])
    print(f"cos_sim_org: {cos_sim_org}, cos_sim_add: {cos_sim_add}")
    topk_sim, topk_ids = torch.topk(l2, 50)
    print(topk_sim)
    print(topk_ids)
    print(tokenizer.decode(topk_ids))

In [306]:
mlm_splade_62783 = Splade(model_pathes["mlm-splade-62783"])
mlm_62783 = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/mlm_model/raw/remove/62783/")
embed_move_w_splade(mlm_62783, mlm_splade_62783)
del mlm_splade_62783
del mlm_62783

path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/
l2: 0.06935861706733704, l2_org: 0.08058971166610718, l2_add: 0.05873292312026024
cos_sim: 0.9980770945549011, cos_sim_org: 0.9979574084281921, cos_sim_add: 0.9981902837753296
tensor([0.4130, 0.3312, 0.3051, 0.3013, 0.2687, 0.2524, 0.2512, 0.2466, 0.2381,
        0.2375, 0.2348, 0.2327, 0.2322, 0.2254, 0.2248, 0.2216, 0.2180, 0.2172,
        0.2168, 0.2167, 0.2157, 0.2105, 0.2081, 0.2072, 0.2043, 0.2040, 0.2029,
        0.2026, 0.2000, 0.1999, 0.1993, 0.1964, 0.1934, 0.1911, 0.1908, 0.1900,
        0.1899, 0.1882, 0.1882, 0.1865, 0.1863, 0.1862, 0.1853, 0.1842, 0.1840,
        0.1836, 0.1836, 0.1835, 0.1825, 0.1824], grad_fn=<TopkBackward0>)
tensor([ 1010,  1996,  1998,  1011,  1012,  2035,  2008,  1999,  2034,  1000,
         2028,  2119,  2036, 29649,

In [307]:
mlm_splade = Splade(model_pathes["mlm-splade-30522"])
mlm_bert = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/mlm_model/raw/remove/30522/")
embed_move_w_splade(mlm_bert, mlm_splade)
del mlm_bert
del mlm_splade

path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/
l2: 0.15539950132369995, l2_org: 0.15539950132369995, l2_add: nan
cos_sim: 0.9933962225914001, cos_sim_org: 0.9933962225914001, cos_sim_add: nan
tensor([0.4178, 0.3455, 0.3240, 0.3098, 0.2900, 0.2883, 0.2878, 0.2853, 0.2830,
        0.2827, 0.2812, 0.2800, 0.2789, 0.2766, 0.2760, 0.2752, 0.2748, 0.2728,
        0.2717, 0.2697, 0.2690, 0.2662, 0.2661, 0.2656, 0.2654, 0.2652, 0.2649,
        0.2636, 0.2633, 0.2632, 0.2626, 0.2624, 0.2623, 0.2615, 0.2607, 0.2598,
        0.2597, 0.2589, 0.2586, 0.2585, 0.2580, 0.2580, 0.2578, 0.2575, 0.2574,
        0.2572, 0.2572, 0.2564, 0.2560, 0.2559], grad_fn=<TopkBackward0>)
tensor([ 1010,  1998,  1011, 29649,  1996,  1000,  1012, 10985, 28075,  2015,
        29655, 23726,  2035,  1999,  7414, 21767,  5933, 29023, 19

In [308]:
splade = Splade(model_pathes["splade"])
bert = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
embed_move_w_splade(bert, splade)
del bert
del splade

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


l2: 0.20133037865161896, l2_org: 0.20133037865161896, l2_add: nan
cos_sim: 0.9893619418144226, cos_sim_org: 0.9893619418144226, cos_sim_add: nan
tensor([0.3735, 0.3532, 0.3423, 0.3410, 0.3384, 0.3377, 0.3334, 0.3330, 0.3289,
        0.3278, 0.3277, 0.3276, 0.3252, 0.3244, 0.3236, 0.3219, 0.3217, 0.3207,
        0.3207, 0.3205, 0.3197, 0.3191, 0.3191, 0.3190, 0.3189, 0.3185, 0.3180,
        0.3167, 0.3165, 0.3144, 0.3130, 0.3127, 0.3121, 0.3119, 0.3117, 0.3116,
        0.3114, 0.3112, 0.3107, 0.3106, 0.3101, 0.3100, 0.3099, 0.3098, 0.3095,
        0.3094, 0.3094, 0.3088, 0.3086, 0.3085], grad_fn=<TopkBackward0>)
tensor([29649, 19591, 26262,  1996, 28079,  5058,  1067,  6120,  1998, 21639,
        23726, 24140,  5567,  9432,  9857, 27717,  1010, 18300, 10372, 22462,
         7451, 12781, 29247,  1000,  5296, 29662, 16662, 29653, 26052,  3758,
        29652,  5378, 20984,  5933,  3386,  2441,  2249,  4052,  4786,  2760,
        24915, 29656,  8168, 20821, 12938,  4793, 16115, 26375,  4036

In [290]:
bert = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")
mlm_bert = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/mlm_model/raw/remove/30522/")
embed_move_w_bert(bert, mlm_bert, tokenizer)
del bert
del mlm_bert

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


l2_org: 0.1896594762802124, l2_add: nan
cos_sim_org: 0.9900968670845032, cos_sim_add: nan
tensor([0.8245, 0.8822, 0.9048, 0.9196, 0.9361, 0.9366, 0.9368, 0.9401, 0.9420,
        0.9433, 0.9434, 0.9450, 0.9467, 0.9469, 0.9472, 0.9478, 0.9481, 0.9482,
        0.9487, 0.9490, 0.9493, 0.9508, 0.9510, 0.9515, 0.9520, 0.9521, 0.9521,
        0.9522, 0.9523, 0.9524, 0.9524, 0.9529, 0.9532, 0.9533, 0.9536, 0.9539,
        0.9542, 0.9542, 0.9543, 0.9544, 0.9548, 0.9557, 0.9563, 0.9565, 0.9567,
        0.9573, 0.9576, 0.9577, 0.9577, 0.9580], grad_fn=<NegBackward0>)
tensor([ 2133,  1529, 21932, 29658,   999,  2010,  2002,  2201,  3140,  6398,
         4316,  5525,  2014,  4098,  4172,  2182, 10061,  2720,  6473,  3117,
         4632,  5043,  6384,     0,  2516,  3906,  3606,  2745, 15008, 25850,
         1079,  2033,  9809,  3611, 11588,  2017,  3931, 15287,  4727,  6574,
         8142,  2282,  7782,  2986,  4267,  4575,  4075,  2396,  6785,  2751])
... …´s´! his he album forced reporter vehicle

In [292]:
ini_62783 = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/init_model/raw/remove/62783/")
mlm_62783 = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/mlm_model/raw/remove/62783/")
tokenizer = AutoTokenizer.from_pretrained("/home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/mlm_model/raw/remove/62783/")
embed_move_w_bert(ini_62783, mlm_62783, tokenizer)
del ini_62783
del mlm_62783

l2_org: 0.1923682689666748, l2_add: 0.2712787985801697
cos_sim_org: 0.9897450804710388, cos_sim_add: 0.9603198766708374
tensor([0.7223, 0.7256, 0.7430, 0.7435, 0.7449, 0.7472, 0.7490, 0.7515, 0.7532,
        0.7561, 0.7609, 0.7617, 0.7648, 0.7676, 0.7707, 0.7709, 0.7722, 0.7731,
        0.7734, 0.7734, 0.7746, 0.7754, 0.7771, 0.7779, 0.7791, 0.7809, 0.7817,
        0.7824, 0.7825, 0.7827, 0.7832, 0.7833, 0.7852, 0.7852, 0.7864, 0.7872,
        0.7874, 0.7874, 0.7875, 0.7889, 0.7892, 0.7893, 0.7894, 0.7901, 0.7902,
        0.7904, 0.7910, 0.7916, 0.7933, 0.7934], grad_fn=<NegBackward0>)
tensor([30525, 30806, 30675, 30729, 34649, 31151, 30660, 31240, 30743, 32271,
        31713, 30809, 60347, 59650, 32793, 31250, 59963, 59553, 33892, 61525,
        58426, 32057, 60199, 32999, 30834, 60231, 32415, 60871, 32579, 57866,
        58037, 59736, 32934, 61184, 61709, 60003, 35333, 62377, 61701, 62121,
        59403, 62778, 33950, 57087, 61130, 59407, 62317, 38682, 50031, 61681])
∼ ␤ ϭ ␣ ∝ ⊙ ϯ ؉ 

In [53]:
all_results = {}
for k, model_path in model_pathes.items():
    results = search_result(model_path)
    all_results[k] = results

path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/802 [00:00<?, ?it/s]

path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/802 [00:00<?, ?it/s]

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/802 [00:00<?, ?it/s]

In [54]:
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'ndcg_cut_10'})
all_ndcg = {}
all_ave_ndcg = {}
for k, result in all_results.items():
    ndcg = evaluator.evaluate(result)
    all_ndcg[k] = ndcg
    all_ave_ndcg[k] = np.mean([val["ndcg_cut_10"] for val in ndcg.values()])

In [55]:
all_ave_ndcg

{'mlm-splade-62783': 0.15577710688764188,
 'mlm-splade-30522': 0.15691002377354152,
 'splade-62783': 0.15610918267920665,
 'splade': 0.1549133613255218}

In [56]:
df_all_ndcg = {}
model_names = ["mlm-splade-62783", "mlm-splade-30522", "splade-62783", "splade"]
for k in model_names:
    target = all_ndcg[k]
    df_all_ndcg[k] = [val['ndcg_cut_10'] for val in target.values()]
    
df_all_ndcg = pd.DataFrame(df_all_ndcg)
df_all_ndcg.head()

,mlm-splade-62783,mlm-splade-30522,splade-62783,splade
0,0.339160,0.213986,0.339160,0.169580
1,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000
3,0.213986,0.169580,0.213986,0.339160
4,0.867044,0.636682,0.636682,0.636682


In [57]:
diff_from_splade = {}
diff_model_names = ["mlm-splade-62783", "mlm-splade-30522"]
q_index = list(queries.keys())
for k in diff_model_names:
    target = all_ndcg[k]
    base = all_ndcg["splade"]
    diff_from_splade[k] = [val1["ndcg_cut_10"] - val2["ndcg_cut_10"] for val1, val2 in zip(target.values(), base.values())]
    
df_diff_from_splade = pd.DataFrame(diff_from_splade)
df_diff_from_splade

,mlm-splade-62783,mlm-splade-30522
0,0.169580,0.044406
1,0.000000,0.000000
2,0.000000,0.000000
3,-0.125174,-0.169580
4,0.230361,0.000000
...,...,...
995,-0.125174,-0.125174
996,0.000000,0.000000
997,-0.098039,0.004058
998,0.000000,0.000000


In [58]:
diff_from_mlm_splade = {}
diff_model_names = ["mlm-splade-62783", "splade"]
q_index = list(queries.keys())
for k in diff_model_names:
    target = all_ndcg[k]
    base = all_ndcg["mlm-splade-30522"]
    diff_from_mlm_splade[k] = [val1["ndcg_cut_10"] - val2["ndcg_cut_10"] for val1, val2 in zip(target.values(), base.values())]
    
df_diff_from_mlm_splade = pd.DataFrame(diff_from_mlm_splade)
df_diff_from_mlm_splade

,mlm-splade-62783,splade
0,0.125174,-0.044406
1,0.000000,0.000000
2,0.000000,0.000000
3,0.044406,0.169580
4,0.230361,0.000000
...,...,...
995,0.000000,0.125174
996,0.000000,0.000000
997,-0.102097,-0.004058
998,0.000000,0.000000


In [59]:
qids_adalm_splade_better = df_diff_from_splade[(df_diff_from_splade["mlm-splade-62783"] > 0.0)].index
qids_adalm_splade_better = [q_index[i] for i in qids_adalm_splade_better]

In [60]:
qids_adalm_splade_worse = df_diff_from_splade[df_diff_from_splade["mlm-splade-62783"] < 0.0].index
qids_adalm_splade_worse = [q_index[i] for i in qids_adalm_splade_worse]

In [61]:
print(len(qrels), len(qids_davosplade_better), len(qids_davosplade_worse))

NameError: name 'qids_davosplade_better' is not defined

In [ ]:
def count_em_w_space_tk(model_name, qids, all_results):
    def encode_doc(model, e_query, text, t_query, t_query_space):
        t_text = model.tokenizer(text, max_length=512, return_tensors="pt")
        with torch.no_grad():
            e_text = model.encode(**t_text)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.1]
            score = score[0][score[0] > 0.1]
            tids = model.tokenizer.decode(ids)
        t_tids = tids.split()
        match_type = {"em": [], "not-em": [], "space-tk": []}
        not_em = []
        for t in t_tids:
            if t in t_query:
                match_type["em"].append(t)
            else:
                match_type["not-em"].append(t)
         
            if t in t_query_space:
                match_type["space-tk"].append(t)
                
        return match_type
    
    model_path = model_pathes[model_name]
    model = Splade(model_path)
    space_tk = BasicTokenizer()
    
    match_token_splade_nums = []
    expand_token_splade_nums = []
    match_token_tk_nums = []
        
    for qid in qids:
        query = queries[qid]
        top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
        t_query_id = model.tokenizer(query, max_length=512, return_tensors="pt")
        t_query = model.tokenizer.tokenize(query)
        t_query_space = space_tk.tokenize(query)
        with torch.no_grad():    
            e_query = model.encode(**t_query_id)

        top_qrels = []
        try:
            q_qrels = qrels[qid]
        except:
            continue
        for i, (cid, qrel) in enumerate(q_qrels.items()):            
            if qrel > 0:
                text = corpus[cid]["title"] + " " + corpus[cid]["text"]
                t_text = model.tokenizer.tokenize(text)
                match_token_tk_nums.append(len([t for t in t_query if t in t_text]))
                match_type = encode_doc(model, e_query, text, t_query, t_query_space)
                match_token_splade_nums.append(len(match_type["em"]))
                expand_token_splade_nums.append(len(match_type["not-em"]))
        

    return match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums

In [ ]:
observe_model = ["mlm-splade-62783", "mlm-splade-30522", "splade-62783", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, qids_adalm_splade_better, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

In [ ]:
observe_model = ["mlm-splade-62783", "mlm-splade-30522", "splade-62783", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, qids_adalm_splade_worse, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

In [ ]:
observe_model = ["mlm-splade-62783", "mlm-splade-30522", "splade-62783", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, q_index, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

In [348]:
ndcg_adalm_splade_better = df_all_ndcg[(df_diff_from_splade["mlm-splade-62783"] > 0.1) & (df_diff_from_mlm_splade["mlm-splade-62783"] > 0.1)]
qids_adalm_splade_better_show = df_diff_from_splade[(df_diff_from_splade["mlm-splade-62783"] > 0.1) & (df_diff_from_mlm_splade["mlm-splade-62783"] > 0.1)].index
qids_adalm_splade_better_show = [q_index[i] for i in qids_adalm_splade_better_show]

In [358]:
ndcg_adalm_splade_worse = df_all_ndcg[(df_diff_from_splade["mlm-splade-62783"] < -0.1) & (df_diff_from_mlm_splade["mlm-splade-62783"] < -0.1)]
qids_adalm_splade_worse_show = df_diff_from_splade[(df_diff_from_splade["mlm-splade-62783"] < -0.1) & (df_diff_from_mlm_splade["mlm-splade-62783"] < -0.1)].index
qids_adalm_splade_worse_show = [q_index[i] for i in qids_adalm_splade_worse_show]

# domain specific words

In [ ]:
def em_w_space_tk(model_name, qids, all_results):
    def encode_doc(model, e_query, text, t_query, t_query_space):
        t_text_id = model.tokenizer(text, max_length=512, return_tensors="pt")
        t_text = model.tokenizer.tokenize(text)
        with torch.no_grad():
            e_text = model.encode(**t_text_id)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.0]
            score = score[0][score[0] > 0.0]
            tids = model.tokenizer.decode(ids)
        t_tids = tids.split()
        match_type = {"em": [], "not-em": [], "space-tk": []}
        not_em = []
        for t in t_tids:
            if t in t_text:
                match_type["em"].append(t)
            else:
                match_type["not-em"].append(t)
         
            if t in t_query_space:
                match_type["space-tk"].append(t)
                
        return match_type
    
    model_path = model_pathes[model_name]
    model = Splade(model_path)
    space_tk = BasicTokenizer()
    
    match_token_splade = []
    expand_token_splade = []
    match_token_tk = []
        
    for qid in qids:
        query = queries[qid]
        top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
        t_query_id = model.tokenizer(query, max_length=512, return_tensors="pt")
        t_query = model.tokenizer.tokenize(query)
        t_query_space = space_tk.tokenize(query)
        with torch.no_grad():    
            e_query = model.encode(**t_query_id)

        top_qrels = []
        try:
            q_qrels = qrels[qid]
        except:
            continue
        for i, (cid, qrel) in enumerate(q_qrels.items()):            
            if qrel > 0:
                text = corpus[cid]["title"] + " " + corpus[cid]["text"]
                t_text = model.tokenizer.tokenize(text)
                # match_token_tk_nums.append(len([t for t in t_query if t in t_text]))
                match_type = encode_doc(model, e_query, text, t_query, t_query_space)
                match_token_splade += match_type["em"]
                expand_token_splade += match_type["not-em"]
        

    return match_token_splade, expand_token_splade

In [33]:
def calc_idf(corpus, tokenizer):
    N = len(corpus)
    idf = defaultdict(float)
    df = Counter()
    for cid, doc in tqdm_notebook(corpus.items()):
        text = doc["title"] + " " + doc["text"]
        t_doc = tokenizer.tokenize(text)
        df.update(list(set(t_doc)))
    
    for v, freq in df.items():
        idf[v] = np.log(N/freq)
        df[v] = freq / N
    return df, idf

In [25]:
data_path = "/home/gaia_data/iida.h/BEIR/datasets/msmarco"
ms_corpus, ms_queries, ms_qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [39]:
vocab_size = 62783
tk_names = ["splade", f"splade-{vocab_size}"]
ms_tk_df = {}
for model_name in tk_names:
    df, idf = calc_idf(ms_corpus, tks[model_name])
    ms_tk_df[model_name] = df

/tmp/ipykernel_4048098/3938769003.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/8841823 [00:00<?, ?it/s]

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [62]:
tk_df = {}
for model_name in tk_names:
    df, idf = calc_idf(corpus, tks[model_name])
    tk_df[model_name] = df

/tmp/ipykernel_4048098/3938769003.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/25657 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25657 [00:00<?, ?it/s]

In [63]:
word_kind_model = {}

for model_name in tk_names:
    word_kind_model[model_name] = {"domain_words": [], "general_words": []}
    for v in tk_df[model_name]:
        if tk_df[model_name][v] > ms_tk_df[model_name].get(v, 0) * 5:
            word_kind_model[model_name]["domain_words"].append(v)
        else:
            word_kind_model[model_name]["general_words"].append(v)

In [64]:
def num_expand_words(model_name, expand_token_splade, word_kind_model):
    domain_words = []
    general_words = []
    if str(vocab_size) in model_name:
        for w in expand_token_splade:
            if w in word_kind_model[f"splade-{vocab_size}"]["domain_words"]:
                domain_words.append(w)
            else:
                general_words.append(w)
    else:
        for w in expand_token_splade:
            if w in word_kind_model["splade"]["domain_words"]:
                domain_words.append(w)
            else:
                general_words.append(w)
                
    print(f"expand-domain_words:{len(domain_words)}, expand-general_words:{len(general_words)}")

In [65]:
observe_model = [f"mlm-splade-{vocab_size}", "mlm-splade-30522", f"splade-{vocab_size}", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_splade, expand_token_splade = em_w_space_tk(model_name, qids_adalm_splade_better, all_results)
    num_expand_words(model_name, expand_token_splade, word_kind_model)

path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


expand-domain_words:11, expand-general_words:8966
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


expand-domain_words:0, expand-general_words:9167
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model_init/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-init_model-raw-remove--62783-batch_size_40-2022-04-17_09-55-08/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


expand-domain_words:7, expand-general_words:12428
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (1817 > 512). Running this sequence through the model will result in indexing errors


expand-domain_words:0, expand-general_words:9838


In [68]:
N = len(qids_adalm_splade_better)
print(11/N, 8996/N)
print(9167/N)
print(7/N, 12428/N)
print(9838/N)

0.052132701421800945 42.63507109004739
43.44549763033175
0.03317535545023697 58.90047393364929
46.62559241706161


In [66]:
observe_model = [f"mlm-splade-{vocab_size}", "mlm-splade-30522", f"splade-{vocab_size}", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_splade, expand_token_splade = em_w_space_tk(model_name, q_index, all_results)
    num_expand_words(model_name, expand_token_splade, word_kind_model)

path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


expand-domain_words:48, expand-general_words:37127
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


expand-domain_words:0, expand-general_words:39386
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model_init/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-init_model-raw-remove--62783-batch_size_40-2022-04-17_09-55-08/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


expand-domain_words:20, expand-general_words:56410
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (783 > 512). Running this sequence through the model will result in indexing errors


expand-domain_words:0, expand-general_words:43078


# show docs

In [392]:
def analysis(model_name, qid, all_results):
    def encode_vec(model, e_query, text):
        t_text = model.tokenizer(text, max_length=512, return_tensors="pt")
        with torch.no_grad():
            e_text = model.encode(**t_text)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.0]
            score = score[0][score[0] > 0.0]
            tids = model.tokenizer.decode(ids)
        print([round(s, 2) for s in score.tolist()])
        print(tids)
        t_tids = tids.split()
        match_type = {"em": [], "not-em": [], "space-tk": []}
        not_em = []
        for t in t_tids:
            if t in t_query:
                match_type["em"].append(t)
            else:
                match_type["not-em"].append(t)
            
            if t in t_query_space:
                match_type["space-tk"].append(t)
                
        print(match_type)
                    
    model_path = model_pathes[model_name]
    query = queries[qid]
    top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
    model = Splade(model_path)
    space_tk = BasicTokenizer()
    t_query_id = model.tokenizer(query, max_length=512, return_tensors="pt")
    t_query = model.tokenizer.tokenize(query)
    t_query_space = space_tk.tokenize(query)
    with torch.no_grad():    
        e_query = model.encode(**t_query_id)
    
    for c_cid, qrel in qrels[qid].items():
        if qrel > 0:
            within_top10 = [(i+1, cid, score) for i, (cid, score) in enumerate(top_10_doc) if cid==c_cid]
            if within_top10:
                print(within_top10, qrel)
            else:
                print("not exist within top10", c_cid)
            text = corpus[c_cid]["title"] + " " + corpus[c_cid]["text"]
            encode_vec(model, e_query, text)
        
    del model

In [394]:
observe_model = ["mlm-splade-62783", "mlm-splade-30522", "splade"]
for qid in qids_adalm_splade_better_show:
    print("----------------------------")
    print(qid, queries[qid])
    for model_name in observe_model:
        print("-------------")
        print(model_name)
        analysis(model_name, qid, all_results)

----------------------------
70 Activation of PPM1D suppresses p53 function.
-------------
mlm-splade-62783
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[(1, '5956380', 28.107315063476562)] 1
[3.31, 3.21, 3.19, 2.06, 1.9, 1.87, 1.55, 1.5, 1.42, 1.41, 1.19, 1.12, 0.97, 0.9, 0.4, 0.39, 0.37, 0.34, 0.25, 0.21]
ppm p53 pp activation tp531 function53 suppress pd 1 suppression dna ppt 53 cancer activated suppressor suppressed
{'em': ['ppm', 'p53', 'activation'], 'not-em': ['pp', 'tp531', 'function53', 'suppress', 'pd', '1', 'suppression', 'dna', 'ppt', '53', 'cancer', 'activated', 'suppressor', 'suppressed'], 'space-tk': ['p53', 'activation']}
not exist within top10 4414547
[3.44, 3.22, 3.1, 1.73, 1.73, 1.58, 1.45, 1.31, 1.27, 1.07, 1.06, 0.87, 0.64, 0.47, 0.46, 0.4, 0.4, 0.29, 0.25, 0.15]
p53 ppm pp p153 suppression tp53d dna 1 suppress control 53 cancer ppt function suppressed effect suppressor
{'em': ['p53', 'ppm', 'function'], 'not-em': ['pp', 'p153', 'suppression', 'tp53d', 'dna', '1', 'suppress', 'control', '53', 'cancer', 'ppt', 'suppressed', 'effect', 'suppressor'], 'space-tk': ['p53', 'function']}
-------------
mlm-splade-30522
path

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[(3, '5956380', 19.199443817138672)] 1
[6.05, 2.92, 2.57, 1.54, 1.34, 1.28, 1.21, 0.94, 0.79, 0.32, 0.2, 0.01]
pp531m p activation suppressd function suppression 1 drug
{'em': ['p', 'activation', 'function'], 'not-em': ['pp531m', 'suppressd', 'suppression', '1', 'drug'], 'space-tk': ['activation', 'function']}
not exist within top10 4414547
[5.78, 3.12, 2.45, 1.7, 1.51, 0.86, 0.49, 0.18, 0.16, 0.16, 0.12, 0.03]
pp531 pmd suppression influence control 1 effect 53
{'em': [], 'not-em': ['pp531', 'pmd', 'suppression', 'influence', 'control', '1', 'effect', '53'], 'space-tk': []}
-------------
splade
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


...

[(10, '16322674', 22.686885833740234)] 1
[3.58, 3.56, 2.33, 2.25, 1.37, 1.14, 1.14, 1.13, 1.11, 0.72, 0.65, 0.63, 0.62, 0.62, 0.47, 0.3, 0.21, 0.17, 0.16, 0.08]
cancer breast birth weight tumor risk breasts women pregnancy weighting risks baby cancers woman breastfeeding cancerous fat chance child increase
{'em': ['cancer', 'breast', 'birth', 'weight', 'women'], 'not-em': ['tumor', 'risk', 'breasts', 'pregnancy', 'weighting', 'risks', 'baby', 'cancers', 'woman', 'breastfeeding', 'cancerous', 'fat', 'chance', 'child', 'increase'], 'space-tk': ['cancer', 'breast', 'birth', 'weight', 'women']}
[(4, '27123743', 25.149755477905273)] 1
[3.89, 3.79, 1.95, 1.92, 1.75, 1.44, 1.26, 1.15, 0.77, 0.68, 0.68, 0.68, 0.66, 0.56, 0.52, 0.46, 0.4, 0.37, 0.36, 0.24]
cancer breast birth weight tumor breasts women risk cancers baby risks cause woman likely high cancerous weighting breastfeeding overweight bmi
{'em': ['cancer', 'breast', 'birth', 'weight', 'women', 'likely'], 'not-em': ['tumor', 'breasts', 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


not exist within top10 16322674
[3.84, 2.98, 2.17, 1.87, 1.58, 1.46, 0.98, 0.95, 0.88, 0.83, 0.64, 0.38, 0.38, 0.37, 0.28, 0.24, 0.22, 0.17, 0.14, 0.14]
breast cancer weight birth risk tumor weigh baby pregnancy woman women influence effect cancers lump breasts percent age during bra
{'em': ['breast', 'cancer', 'weight', 'birth', 'women'], 'not-em': ['risk', 'tumor', 'weigh', 'baby', 'pregnancy', 'woman', 'influence', 'effect', 'cancers', 'lump', 'breasts', 'percent', 'age', 'during', 'bra'], 'space-tk': ['breast', 'cancer', 'weight', 'birth', 'women']}
[(1, '27123743', 25.320796966552734)] 1
[4.19, 3.45, 2.63, 1.79, 1.69, 1.39, 0.81, 0.75, 0.71, 0.64, 0.62, 0.62, 0.61, 0.58, 0.54, 0.49, 0.44, 0.41, 0.28, 0.26]
breast cancer weight birth tumor risk babyweight high cancers women woman cause pregnancy likely influence obesity weigh breasts disease
{'em': ['breast', 'cancer', 'weight', 'birth', 'women', 'likely'], 'not-em': ['tumor', 'risk', 'babyweight', 'high', 'cancers', 'woman', 'caus

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[(8, '16322674', 15.468591690063477)] 1
[3.21, 2.6, 1.93, 1.6, 1.46, 0.89, 0.81, 0.53, 0.51, 0.43, 0.42, 0.33, 0.32, 0.28, 0.23, 0.18, 0.15, 0.09, 0.06, 0.05]
breast cancer weight birth tumor baby woman pregnancy effect risk female born women fat cancers chance obesity breasts positive effects
{'em': ['breast', 'cancer', 'weight', 'birth', 'women'], 'not-em': ['tumor', 'baby', 'woman', 'pregnancy', 'effect', 'risk', 'female', 'born', 'fat', 'cancers', 'chance', 'obesity', 'breasts', 'positive', 'effects'], 'space-tk': ['breast', 'cancer', 'weight', 'birth', 'women']}
[(1, '27123743', 20.315526962280273)] 1
[3.55, 2.94, 1.92, 1.82, 1.63, 0.76, 0.75, 0.69, 0.56, 0.55, 0.49, 0.44, 0.43, 0.39, 0.36, 0.35, 0.33, 0.32, 0.31, 0.31]
breast cancer weight tumor birth baby fat woman obesity high pregnancy cause likely low chance risk female born women cancers
{'em': ['breast', 'cancer', 'weight', 'birth', 'likely', 'women'], 'not-em': ['tumor', 'baby', 'fat', 'woman', 'obesity', 'high', 'pregnanc

In [360]:
observe_model = ["mlm-splade-62783", "mlm-splade-30522", "splade"]
for qid in qids_adalm_splade_worse_show:
    print("----------------------------")
    print(qid, queries[qid])
    for model_name in observe_model:
        print("-------------")
        print(model_name)
        analysis(model_name, qid, all_results)

----------------------------
54 AMP-activated protein kinase (AMPK) activation increases inflammation-related fibrosis in the lungs.
-------------
mlm-splade-62783
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/62783/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--62783-batch_size_40-2022-04-14_08-39-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


not exist within top10 49556906
[4.07, 4.02, 3.74, 2.95, 2.84, 2.49, 1.61, 1.57, 1.19, 0.81, 0.73, 0.67, 0.47, 0.46, 0.42, 0.42, 0.4, 0.38, 0.37, 0.37]
fibrosis amp ampk lungs lung activation amps fi activated activity regulation amplifier inflammatory help cause breathing activations word signaling ␣
{'em': ['fibrosis', 'amp', 'ampk', 'lungs', 'activation', 'activated'], 'not-em': ['lung', 'amps', 'fi', 'activity', 'regulation', 'amplifier', 'inflammatory', 'help', 'cause', 'breathing', 'activations', 'word', 'signaling', '␣'], 'space-tk': ['fibrosis', 'amp', 'ampk', 'lungs', 'activation', 'activated']}
-------------
mlm-splade-30522
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[(6, '49556906', 31.937511444091797)] 1
[6.76, 4.39, 3.59, 3.26, 2.78, 2.21, 2.15, 1.83, 1.35, 0.72, 0.43, 0.41, 0.35, 0.32, 0.31, 0.27, 0.25, 0.21, 0.18, 0.17]
ampbrok lung kinase activation fi lungssis activate protein drug activated k function breath hormone associated benefit enzyme
{'em': ['kinase', 'activation', 'fi', 'protein', 'activated'], 'not-em': ['ampbrok', 'lung', 'lungssis', 'activate', 'drug', 'k', 'function', 'breath', 'hormone', 'associated', 'benefit', 'enzyme'], 'space-tk': ['kinase', 'activation', 'protein', 'activated']}
-------------
splade
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


...

[(2, '27910499', 24.472455978393555)] 1
[3.83, 3.12, 2.68, 2.62, 1.91, 1.79, 1.72, 1.23, 1.12, 1.07, 1.03, 0.67, 0.61, 0.46, 0.15, 0.12]
diversity tcr t transplant after tc cell cells diverse transplantation transplanting donation tcrs transplanted reduced increase
{'em': ['diversity', 'tcr', 't', 'after', 'cells', 'transplantation'], 'not-em': ['transplant', 'tc', 'cell', 'diverse', 'transplanting', 'donation', 'tcrs', 'transplanted', 'reduced', 'increase'], 'space-tk': ['diversity', 'tcr', 't', 'after', 'cells', 'transplantation']}
-------------
mlm-splade-30522
path /home/gaia_data/iida.h/BEIR/model/S2ORC/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-S2ORC-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_40-2022-04-17_10-06-25/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[(1, '27910499', 29.307308197021484)] 1
[4.44, 4.28, 3.49, 3.45, 2.38, 1.81, 1.2, 1.16, 0.95, 0.78, 0.73, 0.65, 0.6, 0.58, 0.47, 0.43, 0.42, 0.33, 0.3, 0.23]
tc diversity transplant t cell after cellsr implant donateation variation diverse graf dna donation surgery biodiversity donor blood
{'em': ['tc', 'diversity', 'transplant', 't', 'after'], 'not-em': ['cell', 'cellsr', 'implant', 'donateation', 'variation', 'diverse', 'graf', 'dna', 'donation', 'surgery', 'biodiversity', 'donor', 'blood'], 'space-tk': ['diversity', 't', 'after']}
-------------
splade
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[(1, '27910499', 22.54183006286621)] 1
[3.9, 3.73, 3.01, 2.88, 1.94, 1.92, 1.1, 0.73, 0.69, 0.68, 0.63, 0.4, 0.16, 0.14, 0.13, 0.13, 0.11, 0.1]
diversity tc t transplant cell afterr cellsation donation implant biodiversity transfert gene surgery improve limit
{'em': ['diversity', 'tc', 't', 'transplant'], 'not-em': ['cell', 'afterr', 'cellsation', 'donation', 'implant', 'biodiversity', 'transfert', 'gene', 'surgery', 'improve', 'limit'], 'space-tk': ['diversity', 't']}


In [395]:
corpus["1215116"]

{'text': 'Over the past two decades there have been significant achievements in the control of a handful of important human tropical infections [1]. These achievements include the substantive reductions in the prevalence and incidence of the so-called neglected diseases such as lymphatic filariasis, onchocerciasis, guinea worm, leprosy, and trachoma (Box 1) [2]. Each of these neglected diseases is a poverty-promoting and often stigmatizing condition occurring primarily in rural areas of low-income countries (Box 2) [3]. They are ancient afflictions, described in the Bible and other ancient texts, which have burdened humanity for millennia [3]. But now, as a result of aggressive regional vertical interventions, there is a possibility that some neglected tropical infections could be eventually controlled to the point of elimination in some areas of endemicity [2–8]. In the case of guinea worm infection, disease eradication might also soon be possible [9]. Box 2. Common Features of the Ne